In [1]:
import sklearn
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
scikit_version = sklearn.__version__

scikit_version

'0.21.3'

In [3]:
sentimental_data = pd.read_csv('datasets/sentimental_analysis_data.csv', 
                               header=None, 
                               names=['Label', 'Text'], 
                               sep='\t')

sentimental_data.sample(10)

,Label,Text
399,1,Love luv lubb the Da Vinci Code!
5242,0,I hate Harry Potter.
782,1,"The Da Vinci Code was awesome, I can't wait to..."
3198,1,man i loved brokeback mountain!
5780,0,"Always knows what I want, not guy crazy, hates..."
92,1,Da Vinci Code is awesome!
6890,0,", she helped me bobbypin my insanely cool hat ..."
1998,1,I love the shoes and the Harry Potter picture.
4361,0,"Da Vinci Code = Up, Up, Down, Down, Left, Righ..."
482,1,"The Da Vinci Code was awesome, I can't wait to..."


In [4]:
sentimental_data.shape

(6918, 2)

In [5]:
X = sentimental_data['Text']

Y = sentimental_data['Label']

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [7]:
x_train.shape, x_test.shape

((5534,), (1384,))

In [8]:
y_train.shape, y_test.shape

((5534,), (1384,))

In [9]:
tfidf_vect = TfidfVectorizer(max_features=15)

x_trans = tfidf_vect.fit_transform(x_train)

In [10]:
tfidf_vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=15,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [11]:
print(x_trans[0:3])

  (0, 0)	0.6005395584487301
  (0, 5)	0.5653990797384641
  (0, 11)	0.5653990797384641
  (1, 12)	0.6313555492455504
  (1, 9)	0.4488584195469285
  (1, 6)	0.4488584195469285
  (1, 1)	0.44546875181152307
  (2, 12)	0.7192272641186084
  (2, 4)	0.4011655534639527
  (2, 13)	0.4011655534639527
  (2, 3)	0.4010542855728917


In [12]:
x_trans.shape

(5534, 15)

In [13]:
classifier = LinearSVC(C=1.0, max_iter=1000, tol=1e-3)
linear_svc_model = classifier.fit(x_trans, y_train)

linear_svc_model

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.001,
          verbose=0)

In [14]:
x_test_trans = tfidf_vect.fit_transform(x_test)

In [15]:
x_test_trans.shape

(1384, 15)

In [16]:
y_pred = linear_svc_model.predict(x_test_trans)

y_pred

array([1, 1, 0, ..., 1, 1, 0])

In [17]:
pred_results = pd.DataFrame({'y_test': y_test,
                             'y_pred': y_pred})

pred_results.sample(5)

,y_test,y_pred
1538,1,1
3555,1,1
313,1,1
4108,0,0
3508,1,0


In [18]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8901734104046243

In [19]:
text_clf_param = {}

text_clf_param['preprocessing'] = tfidf_vect
text_clf_param['model'] = linear_svc_model
text_clf_param['sklearn_version'] = scikit_version
text_clf_param['accuracy'] = accuracy

In [20]:
text_clf_param

{'preprocessing': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.float64'>, encoding='utf-8',
                 input='content', lowercase=True, max_df=1.0, max_features=15,
                 min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                 smooth_idf=True, stop_words=None, strip_accents=None,
                 sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, use_idf=True, vocabulary=None),
 'model': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
           intercept_scaling=1, loss='squared_hinge', max_iter=1000,
           multi_class='ovr', penalty='l2', random_state=None, tol=0.001,
           verbose=0),
 'sklearn_version': '0.21.3',
 'accuracy': 0.8901734104046243}

In [21]:
import joblib

In [22]:
filename = 'models/text_clf_checkpoint.joblib'

In [23]:
joblib.dump(text_clf_param, filename)

['models/text_clf_checkpoint.joblib']

In [24]:
clf_checkpoint = joblib.load(filename)

In [25]:
reloaded_vect = clf_checkpoint['preprocessing']

reloaded_vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=15,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [26]:
clf_model = clf_checkpoint['model']

clf_model

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.001,
          verbose=0)

In [27]:
x_test_trans_new = reloaded_vect.fit_transform(x_test)

In [28]:
y_pred = clf_model.predict(x_test_trans_new)

y_pred

array([1, 1, 0, ..., 1, 1, 0])

In [29]:
accuracy_score(y_test, y_pred)

0.8901734104046243

In [30]:
clf_checkpoint['accuracy']

0.8901734104046243

In [31]:
from sklearn.pipeline import Pipeline

In [32]:
clf_pipeline = Pipeline(steps=[('tfidf_vect', tfidf_vect), ('classifier', classifier)])

pipeline_model = clf_pipeline.fit(x_train, y_train)

In [33]:
pipeline_model

Pipeline(memory=None,
         steps=[('tfidf_vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=15,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
     

In [34]:
y_pred = pipeline_model.predict(x_test)

In [35]:
accuracy = accuracy_score(y_test, y_pred)

accuracy

0.8901734104046243

In [36]:
pipe_clf_param = {}

pipe_clf_param['pipeline_clf'] = pipeline_model
pipe_clf_param['sklearn_version'] = scikit_version
pipe_clf_param['accuracy'] = accuracy

In [37]:
pipe_clf_param

{'pipeline_clf': Pipeline(memory=None,
          steps=[('tfidf_vect',
                  TfidfVectorizer(analyzer='word', binary=False,
                                  decode_error='strict',
                                  dtype=<class 'numpy.float64'>,
                                  encoding='utf-8', input='content',
                                  lowercase=True, max_df=1.0, max_features=15,
                                  min_df=1, ngram_range=(1, 1), norm='l2',
                                  preprocessor=None, smooth_idf=True,
                                  stop_words=None, strip_accents=None,
                                  sublinear_tf=False,
                                  token_pattern='(?u)\\b\\w\\w+\\b',
                                  tokenizer=None, use_idf=True,
                                  vocabulary=None)),
                 ('classifier',
                  LinearSVC(C=1.0, class_weight=None, dual=True,
                            fit_intercept

In [38]:
filename = 'models/pipe_clf_checkpoint.joblib'

In [39]:
joblib.dump(pipe_clf_param, filename)

['models/pipe_clf_checkpoint.joblib']

In [40]:
pipe_clf_checkpoint = joblib.load(filename)

In [41]:
reloaded_pipeline = pipe_clf_checkpoint['pipeline_clf']

reloaded_pipeline

Pipeline(memory=None,
         steps=[('tfidf_vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=15,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
     

In [42]:
y_pred = reloaded_pipeline.predict(x_test)

In [43]:
accuracy_score(y_test, y_pred)

0.8901734104046243

In [44]:
pipe_clf_checkpoint['accuracy']

0.8901734104046243